# L1: NLP tasks with a simple interface 🗞️

Load your HF API key and relevant Python libraries.

In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [2]:
# Helper function
import requests, json

#Summarization endpoint
def get_completion(inputs, parameters=None,ENDPOINT_URL=os.environ['HF_API_SUMMARY_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",
                                ENDPOINT_URL, headers=headers,
                                data=json.dumps(data)
                               )
    return json.loads(response.content.decode("utf-8"))

### How about running it locally?
The code would look very similar if you were running it locally instead of from an API. The same is true for all the models in the rest of the course, make sure to check the [Pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines) documentation page

```py
from transformers import pipeline

get_completion = pipeline("summarization", model="shleifer/distilbart-cnn-12-6")

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
```

## Building a text summarization app

In [3]:
text = ('''Generative AI (GenAI) is a powerful type of artificial intelligence capable of creating new 
        content—including text, images, code, and music—rather than just classifying or analyzing 
        existing data. At its core are Large Language Models (LLMs) and other deep learning 
        architectures trained on vast datasets. These models learn patterns and structures, allowing 
        them to produce outputs that are remarkably original and contextually relevant. The applications 
        of GenAI span industries, from automating content creation and accelerating scientific discovery 
        to revolutionizing software development and personalized customer experiences. Its rapid 
        evolution marks a significant shift, raising important questions about creativity, ethics, and the 
        future of work.''')

get_completion(text)

[{'summary_text': ' Generative AI (GenAI) is a powerful type of artificial intelligence capable of creating new content . At its core are Large Language Models (LLMs) and other deep learning architectures trained on vast datasets . Applications span industries, from automating content creation and accelerating scientific discovery to revolutionizing software development and personalized customer experiences .'}]

### Getting started with Gradio `gr.Interface` 

In [4]:
import gradio as gr
def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']
    
gr.close_all()
demo = gr.Interface(fn=summarize, inputs="text", outputs="text")
demo.launch(share=True, server_port=int(os.environ['PORT1']))

/usr/local/lib/python3.9/site-packages/gradio_client/documentation.py:105: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/usr/local/lib/python3.9/site-packages/gradio_client/documentation.py:105: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


Running on local URL:  https://0.0.0.0:7860
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio


`demo.launch(share=True)` lets you create a public link to share with your team or friends.

In [5]:
import gradio as gr

def summarize(input):
    output = get_completion(input)
    return output[0]['summary_text']

gr.close_all()
demo = gr.Interface(fn=summarize, 
                    inputs=[gr.Textbox(label="Text to summarize", lines=6)],
                    outputs=[gr.Textbox(label="Result", lines=3)],
                    title="Text summarization with distilbart-cnn",
                    description="Summarize any text using the `shleifer/distilbart-cnn-12-6` model under the hood!"
                   )
demo.launch(share=True, server_port=int(os.environ['PORT2']))

Closing server running on port: 7860
Running on local URL:  https://0.0.0.0:7861
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio


## Building a Named Entity Recognition app

We are using this [Inference Endpoint](https://huggingface.co/inference-endpoints) for `dslim/bert-base-NER`, a 108M parameter fine-tuned BART model on the NER task.

### How about running it locally?

```py
from transformers import pipeline

get_completion = pipeline("ner", model="dslim/bert-base-NER")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}
    
```

In [6]:
API_URL = os.environ['HF_API_NER_BASE'] #NER endpoint
text = "My name is Sethukkarasi, I'm building DeepLearningAI and I live in Chennai"
get_completion(text, parameters=None, ENDPOINT_URL= API_URL)

[{'entity': 'B-PER',
  'score': 0.9981382,
  'index': 4,
  'word': 'Seth',
  'start': 11,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.99270904,
  'index': 5,
  'word': '##uk',
  'start': 15,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.9909546,
  'index': 6,
  'word': '##kara',
  'start': 17,
  'end': 21},
 {'entity': 'I-PER',
  'score': 0.802605,
  'index': 7,
  'word': '##si',
  'start': 21,
  'end': 23},
 {'entity': 'B-ORG',
  'score': 0.9913073,
  'index': 13,
  'word': 'Deep',
  'start': 38,
  'end': 42},
 {'entity': 'I-ORG',
  'score': 0.99574995,
  'index': 14,
  'word': '##L',
  'start': 42,
  'end': 43},
 {'entity': 'I-ORG',
  'score': 0.9956929,
  'index': 15,
  'word': '##ear',
  'start': 43,
  'end': 46},
 {'entity': 'I-ORG',
  'score': 0.9956085,
  'index': 16,
  'word': '##ning',
  'start': 46,
  'end': 50},
 {'entity': 'I-ORG',
  'score': 0.9060424,
  'index': 17,
  'word': '##A',
  'start': 50,
  'end': 51},
 {'entity': 'I-ORG',
  'score': 0.6394908,
  'index': 1

#### gr.interface()
- Notice below that we pass in a list `[]` to `inputs` and to `outputs` because the function `fn` (in this case, `ner()`, can take in more than one input and return more than one output.
- The number of objects passed to `inputs` list should match the number of parameters that the `fn` function takes in, and the number of objects passed to the `outputs` list should match the number of objects returned by the `fn` function.

In [7]:
def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    return {"text": input, "entities": output}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Sethukkarasi and I live in Chennai", "My name is Elan and work at Pondicherry"])
demo.launch(share=True, server_port=int(os.environ['PORT3']))

Closing server running on port: 7860
Closing server running on port: 7861
Running on local URL:  https://0.0.0.0:7862
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio


### Adding a helper function to merge tokens

In [8]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity'].startswith('I-') and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens

def ner(input):
    output = get_completion(input, parameters=None, ENDPOINT_URL=API_URL)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()
demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=2)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    examples=["My name is Sethukkarasi, I'm building DeeplearningAI and I live in Chennai", "My name is Elan, I live in Pondicherry and work at HuggingFace"])

demo.launch(share=True, server_port=int(os.environ['PORT4']))

Closing server running on port: 7860
Closing server running on port: 7862
Closing server running on port: 7861
Running on local URL:  https://0.0.0.0:7863
IMPORTANT: You are using gradio version 3.37.0, however version 4.44.1 is available, please upgrade.
--------

Could not create share link. Missing file: /usr/local/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_linux_amd64
2. Rename the downloaded file to: frpc_linux_amd64_v0.2
3. Move the file to this location: /usr/local/lib/python3.9/site-packages/gradio


In [9]:
gr.close_all()

Closing server running on port: 7860
Closing server running on port: 7862
Closing server running on port: 7863
Closing server running on port: 7861


## How to get your own Hugging Face API key (token)

Hugging Face "API keys" are called "User Access tokens".  

You can create your own User Access Tokens here: [Access Tokens](https://huggingface.co/settings/tokens).

#### Save your user access tokens to environment variables
To save your access token securely on your own machine:
- Create a `.env` file in the root directory of your project.
- Edit the file to contain the following:  
`HF_API_KEY="abc123"` replace that string with your user access token.
- Save the .env file.
- Install Python-dotenv, which allows you to run that first code cell at the top of this jupyter notebook:  
`pip install python-dotenv`


For more information on how to get your own access tokens, please see [User access tokens](https://huggingface.co/docs/hub/security-tokens#:~:text=To%20create%20an%20access%20token,you're%20ready%20to%20go!)